Housing prices prediction - Barcelona
==================

### Downloading data from website

Download data from idealista and save it as a mongodb database.

In [12]:
from selenium import webdriver
browser = webdriver.Firefox()

def pageFormat(start,end,page):
    res = "https://www.idealista.com/venta-viviendas/barcelona-barcelona/"
    res += "con-precio-hasta_{},precio-desde_{}/{}".format(
        end,start, "pagina-{}.html".format(page) if page>1 else "")
    return res




https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_80000,precio-desde_60000/?ordenado-por=fecha-publicacion-asc


In [30]:
from lxml import html
from lxml.cssselect import CSSSelector
import time

def loadPage(page):
    browser.get(page)
    return browser.page_source

def loadPageTree(page_source):
    tree = html.document_fromstring(page_source)
    return tree

def getListOfHouses(tree):
    houses = [elem.get("href") for elem in tree.cssselect('.item-link')]
    return houses
    
def isLastPage(tree):
    return tree.cssselect(".icon-arrow-right-after") == []
    
price_to = 80000
price_from = 60000
page_number = 1
page = pageFormat(price_from,price_to,page_number)


In [86]:
id = "87428761" 
houseurl = "/inmueble/"+id
p = loadPage("https://www.idealista.com/"+houseurl)
t = loadPageTree(p)
t 

 

<Element html at 0x7f8bae2e4048>

In [94]:
def getHouseData(tree):
    data = {}
    price = tree.cssselect(".info-data-price")
    data["price"] = price[0].text_content() if price else ""
    features = tree.cssselect(".info-features")
    data["features"] = [ it.text_content() for idx,it in enumerate(features[0].cssselect("span")) if idx%2==0]  if features else []
    descr = tree.cssselect(".adCommentsLanguage")
    data["description"] = descr[0].text_content() if descr else ""
    details = t.cssselect(".details-property_features")
    print(len(details))
    if details:
        details_house = [ it.text_content() for it in details[0].cssselect("li")] 
        if len(details)>1:
            bdata = [d.cssselect("li") for d in details[1:]] 
            details_building = [ it.text_content() for d in bdata for it in d] 
        else:
            details_building = []
        energy = details[0].cssselect("span") 
        energy_class = energy[0].get("title") if energy else ""
    else:
        details_house = []
        details_building = []
        energy_class = ""
    data["details_house"] = details_house
    data["details_building"] = details_building
    data["energy_class"] = energy_class
    address = t.cssselect("#headerMap")
    address_details = [it.text_content() for it in address[0].cssselect("li")][0:3] if address else []
    data["address"] = address_details
    
    return data
getHouseData(t)

3


{'price': '1.550.000 €',
 'features': [' 218 m² ',
  ' 5 hab. ',
  ' 1ª planta con ascensor ',
  ' Garaje incluido '],
 'description': ' "En el exclusivo barrio de Turó Parc, en una de las más prestigiosas calles, encontramos este maravilloso piso de 185 m2 con 110 m2 de terraza privativa y con salida a jardín comunitario. La finca fue construida por el prestigioso arquitecto José Antonio Coderch en 1960 y conserva en el interior la distribución, así como los elementos originales del arquitecto. Accediendo a la vivienda a través del ascensor principal, entramos directamente en la vivienda. El vestíbulo de entrada conecta las tres zonas, la zona de día, la zona de noche y la zona de servicio con un segundo vestíbulo con acceso al ascensor de servicio y escalera. En la zona de día encontramos el salón comedor con salida a una impresionante terraza de más de 100 m2 que conecta con la terraza comunitaria, el jardín comunitario y la piscina comunitaria, con vestuarios y baño completo. La co